# Models used
1. llama3-70b-8192: Newest Meta LLM with solid Arabic ability
2. deepseek-r1-distill-llama-70b: Strong multilingual fine-tuning
3. gemma2-9b-it: Google’s open multilingual LLM
4. allam-2-7b: Arabic-native model (from KSA)
5. qwen-qwq-32b: High-quality multilingual model

In [ ]:
# API keys for the 5 models
# Replace with your own API keys
llama3_API_KEY = None
gemma2_API_KEY = None
allam_API_KEY = None
qwen_API_KEY = None
openai_API_KEY = None

# Excel Functions

In [10]:
from openpyxl import load_workbook

def extract_questions_and_answers(file_path, question_col=3, answer_col=5, start_row=3):
    wb = load_workbook(file_path)
    ws = wb.active

    questions = []
    gold_answers = []

    row = start_row  # Start for questions
    while True:
        q_cell = ws.cell(row=row, column=question_col)
        a_cell = ws.cell(row=row, column=answer_col)

        # Stop when we reach an empty question cell
        if not q_cell.value:
            break

        questions.append(q_cell.value)
        gold_answers.append(a_cell.value)
        row += 1

    return questions, gold_answers

def write_answers(file_path, answers, output_col=6, start_row=3):
    wb = load_workbook(file_path)
    ws = wb.active

    for i, answer in enumerate(answers):
        row = start_row + i
        ws.cell(row=row, column=output_col, value=answer)
    wb.save(file_path)

In [15]:
file_path = "Arabic_QA_Benchmark_deepseek.xlsx"
questions, answers = extract_questions_and_answers(file_path)

for q, a in zip(questions, answers):
    print("Q:", q)
    print("A:", a)
    print("———")

Q: ما هي أكبر قارة في العالم من حيث المساحة؟
A: قارة آسيا
———
Q: ما هو أطول نهر في العالم؟
A: نهر النيل
———
Q: ما هي الدول التي تطل على البحر الأبيض المتوسط؟
A: في أوروبا:
إسبانيا
فرنسا
موناكو
إيطاليا
سلوفينيا
كرواتيا
البوسنة والهرسك
الجبل الأسود
ألبانيا
اليونان
قبرص (جغرافياً في آسيا لكنها غالباً تُدرج أوروبياً)

 في آسيا:
تركيا
سوريا
لبنان
فلسطين (قطاع غزة يطل على البحر)

 في أفريقيا:
مصر
ليبيا
تونس
الجزائر
المغرب
———
Q: ما هي دول شمال أفريقيا؟
A: مصر
ليبيا
تونس
الجزائر
المغرب
موريتانيا (غالبًا تُدرج ضمن شمال غرب أفريقيا)
———
Q: يعني ايه جغرافيا وبتهتم بدراسة ايه
A: كلمة "جغرافيا" معناها ببساطة علم بيدرس الأرض، وبيوصف كل حاجة عليها، سواء طبيعية زي الجبال والبحار، أو بشرية زي المدن وعدد السكان.

الجغرافيا بتهتم بإيه؟
أول حاجة: الجغرافيا الطبيعية 🌋
ودي بتركز على الحاجات اللي ربنا خلقها في الطبيعة، زي:

الجبال والهضاب والأنهار

البحر والمحيط

المناخ (الجو)

الزلازل والبراكين

تاني حاجة: الجغرافيا البشرية 🏙️
ودي بتهتم باللي بيعمله الإنسان على الأرض، زي:

توزيع الناس في الأماكن

الزراعة و

# Getting the LLMs answers

## 1- llama3-70b-8192 Answers

In [7]:
import os
from groq import Groq
import time

llama3_answers = []
client = Groq(api_key=llama3_API_KEY,)

for i, q in enumerate(questions):
    print(f"Q{i+1}:", q)

    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "أجب بنفس لهجة السؤال، دون الترجمة أو التفسير، وبنفس اللغة.",
                },
                {
                    "role": "user",
                    "content": q,
                }
            ],
            model="llama3-70b-8192",
        )

        answer = chat_completion.choices[0].message.content.strip()
        llama3_answers.append(answer)
        print("A:", answer)

    except Exception as e:
        llama3_answers.append("<<Error: Failed to get response>>")
        print("Error:", e)

    print("———")
    time.sleep(2)  # Prevent hitting RPM limits

Q1: ما هي أكبر قارة في العالم من حيث المساحة؟
A: 亚细亚 هي أكبر قارة في العالم من حيث المساحة.
———
Q2: ما هو أطول نهر في العالم؟
A: نيل!
———
Q3: ما هي الدول التي تطل على البحر الأبيض المتوسط؟
A: تونس، الجزائر، المغرب، مصر، ليبيا، السودان، إيطاليا، اليونان، تركيا، إسرائيل، قبرص، سوريا، لبنان.
———
Q4: ما هي دول شمال أفريقيا؟
A: دول شمال أفريقيا هي المغرب والجزائر وتونس وليبيا ومصر.
———
Q5: يعني ايه جغرافيا وبتهتم بدراسة ايه
A: جغرافيا بتهتم بدراسة الأرض والظواهر الطبيعيةوالانسانية الموجودة عليها، زي الاودية والجبال والبحار والمحيطات والشعوب واللغات والثقافات والمدن والقرى والطرق والاتجاهات والبيئة والاقتصاد والسياسة وغيرها. يعني بتهتم بدراسة كل ما يخص الأرض والبشر عليها.
———
Q6: في أي عام بدأت الثورة الفرنسية؟
A: 1789.
———
Q7: ما هي مدة الحرب العالمية الثانية؟
A: مدته منذ 1 سبتمبر 1939 وحتى 2 سبتمبر 1945.
———
Q8: لماذا سُميت الإمبراطورية البريطانية بالإمبراطورية التي لا تغيب عنها الشمس؟
A: لما أن الإمبراطورية البريطانية كانت تمتد عبر نصف الكرة الأرضية، فكانت الشمس لا تغيب عنها في أي وقت من 

In [8]:
write_answers(file_path, llama3_answers, output_col=6, start_row=3)

## gemma2-9b-it answers

In [12]:
import os
from groq import Groq
import time

gemma2_answers = []
client = Groq(api_key=gemma2_API_KEY,)

for i, q in enumerate(questions):
    print(f"Q{i+1}:", q)

    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "أجب بنفس لهجة السؤال، دون الترجمة أو التفسير، وبنفس اللغة.",
                },
                {
                    "role": "user",
                    "content": q,
                }
            ],
            model="gemma2-9b-it",
        )

        answer = chat_completion.choices[0].message.content.strip()
        gemma2_answers.append(answer)
        print("A:", answer)

    except Exception as e:
        gemma2_answers.append("<<Error: Failed to get response>>")
        print("Error:", e)

    print("———")
    time.sleep(2)  # Prevent hitting RPM limits

Q1: ما هي أكبر قارة في العالم من حيث المساحة؟
A: آسيا
———
Q2: ما هو أطول نهر في العالم؟
A: النيل
———
Q3: ما هي الدول التي تطل على البحر الأبيض المتوسط؟
A: أسبانيا، البرتغال، إيطاليا، فرنسا، موناكو، اليونان، تركيا، لبنان، سورية، إسرائيل، قبرص، الجزائر، تونس، ليبيا.
———
Q4: ما هي دول شمال أفريقيا؟
A: مصر، ليبيا، تونس، الجزائر، المغرب، موريتانيا، مالاوي، غامبيا
———
Q5: يعني ايه جغرافيا وبتهتم بدراسة ايه
A: الجغرافيا دراسة الأرض، شاملًا، من تضاريسها وبيئتها، وأحوال السكان فيها. 🌎🧑‍✈️🐾
———
Q6: في أي عام بدأت الثورة الفرنسية؟
A: 1789
———
Q7: ما هي مدة الحرب العالمية الثانية؟
A: من عام 1939 إلى عام 1945.
———
Q8: لماذا سُميت الإمبراطورية البريطانية بالإمبراطورية التي لا تغيب عنها الشمس؟
A: بسبب امتداد أراضيها لدرجة أن قطاعات منها تقع في دائرتي الضوء و دائرة الظل في كل وقت من اليوم.
———
Q9: أسباب فشل الحملة الفرنسية علي مصر؟
A: عدة أسباب أدت إلى فشل الحملة الفرنسية على مصر،  منها:

*  التحدّيات اللوجستية والبيئية. 
*  الأمراض، وخاصة الحمى. 
*  المقاومة المصرية بقيادة محمد علي. 
*  العوامل  الدي

In [14]:
file_path = "Arabic_QA_Benchmark_gemma.xlsx"
write_answers(file_path, gemma2_answers, output_col=6, start_row=3)

In [ ]:
import os
from groq import Groq
import time

deepseek_answers = []
client = Groq(api_key=deepseek_API_KEY,)

for i, q in enumerate(questions):
    print(f"Q{i+1}:", q)

    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "أجب بنفس لهجة السؤال، دون الترجمة أو التفسير، وبنفس اللغة.",
                },
                {
                    "role": "user",
                    "content": q,
                }
            ],
            model="gemma2-9b-it",
        )

        answer = chat_completion.choices[0].message.content.strip()
        deepseek_answers.append(answer)
        print("A:", answer)

    except Exception as e:
        deepseek_answers.append("<<Error: Failed to get response>>")
        print("Error:", e)

    print("———")
    time.sleep(2)  # Prevent hitting RPM limits

In [ ]:
import openai

gpt4_answers = []
client = openai.OpenAI(api_key=openai_API_KEY)

for q in questions:
    print("Q:", q)
    try:
        response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "Answer in the same dialect and language as the question."},
            {"role": "user", "content": q},
        ]
    )
        answer = response.choices[0].message.content
        gpt4_answers.append(answer)

    except Exception as e:
        gpt4_answers.append("<<Error: Failed to get response>>")
        print("Error:", e)
    
    print("A:", answer)
    print("———")

Q: ما هي أكبر قارة في العالم من حيث المساحة؟
A: أكبر قارة في العالم من حيث المساحة هي آسيا.
———
Q: ما هو أطول نهر في العالم؟
A: أطول نهر في العالم هو نهر النيل، اللي بيجري في شمال شرق أفريقيا. بيمتد لحوالي 6,650 كيلومتر.
———
Q: ما هي الدول التي تطل على البحر الأبيض المتوسط؟
A: الدول اللي تطل على البحر الأبيض المتوسط كتيرة ومنها:

1. مصر
2. ليبيا
3. تونس
4. الجزائر
5. المغرب
6. إسبانيا
7. فرنسا
8. موناكو
9. إيطاليا
10. سلوفينيا
11. كرواتيا
12. البوسنة والهرسك
13. مونتينيغرو
14. ألبانيا
15. اليونان
16. تركيا
17. سوريا
18. لبنان
19. إسرائيل
20. فلسطين (قطاع غزة)
21. قبرص

هذول هم أهم الدول اللي عندها سواحل على البحر الأبيض المتوسط.
———
Q: ما هي دول شمال أفريقيا؟
A: دول شمال أفريقيا تتضمن مصر، ليبيا، تونس، الجزائر، والمغرب. وأحيانًا بيتم إضافة الصحراء الغربية والسودان كجزء من منطقة شمال أفريقيا بناءً على التعريفات الجغرافية.
———
Q: يعني ايه جغرافيا وبتهتم بدراسة ايه
A: يعني جغرافيا، هي علم بيهتم بدراسة الأرض والظواهر الطبيعية والبشرية اللي عليها. بتشمل دراسة توزيع الموارد، السكان، المناخ، 

In [19]:
file_path = "Arabic_QA_Benchmark_GPT.xlsx"
write_answers(file_path, gpt4_answers, output_col=6, start_row=3)